In [1]:
import numpy as np
import pandas as pd

In [2]:
data_f = np.array([[1,0,0,1,0],
                 [1,1,0,0,0],
                 [1,0,0,1,1],
                 [1,1,1,0,0]])
data_g= np.array([[1,0,0,0,0],
                [1,1,1,1,0],
                [0,0,1,1,1],
                [1,1,0,0,0]])

In [3]:
print(len(data_g))

4


In [4]:
#remove duplicate rows
unique_f = np.unique(data_f,axis=0)
unique_f
unique_g = np.unique(data_g,axis=0)
unique_g

array([[0, 0, 1, 1, 1],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 1, 1, 0]])

In [5]:
col_sum = np.sum(data_g, axis=1)
sort_index = np.argsort(col_sum) # index sort in descending order
out_matrix = data_g[:, sort_index]
sort_index

array([0, 3, 2, 1], dtype=int64)

In [6]:
#sort the array by sum(max to min)
data_g=sorted(data_g, key=lambda x: sum(x),reverse=True)
data_g = np.asarray(data_g)
data_g

array([[1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [7]:
#remove super set
for i in range(len(data_g)):
    zero_index = np.where(data_g[i] == 0)[0]
    if(len(zero_index) != 0):
        zero_cols = data_g[:, np.array(zero_index)]
        rows_sum = np.sum(zero_cols, axis=1)
        #print(zero_cols)
        #print(rows_sum)
        if 0 in rows_sum:
            np.delete(data_g,i,0)         

In [8]:
#sum the rows
f=pd.DataFrame(unique_f)
f.loc['Column_Total']= f.sum(numeric_only=True, axis=0)
max_sum_column_f=f.max().max()
f.loc[:,'Row_Total'] = f.sum(numeric_only=True,axis=1)
#df['sum_row'] = df.sum(axis=1)
#df['sum_column'] = df.sum(axis=0)
print(f)
print("the variable that appears max in f is： ", max_sum_column_f)

g=pd.DataFrame(unique_g)
g.loc['Column_Total']= g.sum(numeric_only=True, axis=0)
max_sum_column_g=g.max().max()
g.loc[:,'Row_Total'] = g.sum(numeric_only=True,axis=1)
#df['sum_row'] = df.sum(axis=1)
#df['sum_column'] = df.sum(axis=0)
print(g)
print("the variable that appears max in g is： ", max_sum_column_g)

              0  1  2  3  4  Row_Total
0             1  0  0  1  0          2
1             1  0  0  1  1          3
2             1  1  0  0  0          2
3             1  1  1  0  0          3
Column_Total  4  2  1  2  1         10
the variable that appears max in f is：  4
              0  1  2  3  4  Row_Total
0             0  0  1  1  1          3
1             1  0  0  0  0          1
2             1  1  0  0  0          2
3             1  1  1  1  0          4
Column_Total  3  2  2  2  1         10
the variable that appears max in g is：  3


In [9]:
# pre-req
#!! f and g are dataframes 
check_p=1
satisfy=False
def check_pre(f,g):
# 1. assume var in f and g are the same, but we check if # of columns in f = # of columns in g 
    if len(f) != len(g): 
        print("not dual")
        check_p = 0
# 2.check if largest of sum_row in f <= # of rows in g, largest value of sum_row in g <= # of rows in f 
    if max(f['Row_Total'][0:-1]) > len(g): #f.loc
        print("not dual")
        check_p = 0
    if max(g['Row_Total'][0:-1]) > len(f):
        print("not dual")
        check_p = 0
# 4. sum 2^(-|c in f|)+sum 2^(-|c in g|) >=1, where c is each sum_row value(for loop)>=1
    sum_fg=0
    for i in range (len(f)-1):
        sum_fg+=1/(2**(f['Row_Total'][i]))
    for i in range (len(f)-1):
        sum_fg+=1/(2**(g['Row_Total'][i])) 
    if sum_fg < 1:
        print("not dual")
        check_p = 0
    
# 5. if (# of rows in f) * (# of row in g) >1
    if len(f)*len(g) <= 1:
        print("not dual")
        
# 6. C ^ C' != empty
    for i in range(len(f)):
        if f.loc['Column_Total'][i] ^ g.loc['Column_Total'][i] == True:
            pass
        else:
            satisfy=True
            break
    if satisfy == False:
            print("not dual ")

In [14]:
# test for split function  
data_f = np.array([[1,0,0,1,0],
                 [1,1,0,0,0],
                 [1,0,0,1,1],
                 [0,1,1,0,0]])

f0,f1=split(data_f)
f1

array([[1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

In [ ]:
# #test fk
# data_f = np.array([[1,0,0,1,0],
#                  [1,1,0,0,0],
#                  [1,0,0,1,1],
#                  [0,1,1,0,0]])
# data_g= np.array([[1,1,0,0,0],
#                 [1,1,1,1,0],
#                 [0,0,1,1,1],
#                 [1,1,0,0,0]])
# path=[]
# fk(data_f, data_g, [])

In [12]:
#test for dual printer
dual_matrix=[]
def print_dual(L,R):
    if len(L)==1:
        dual_matrix=np.zeros((len(f.columns)-1,len(f.columns)-1))      
        for index, i in enumerate(f.iloc[0]):
            if i==1:
                dual_matrix[index][index]=1
        return dual_matrix
f=f.head(1)
print(f)
print_dual(f,g)

   0  1  2  3  4  Row_Total
0  1  0  0  1  0          2


array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0.]])

In [13]:
#spiltting, max sum_column variable is the splitting var x
#F0=rows that contains x, change the position of x to 0
#F1=http://localhost:8888/notebooks/research%20.ipynb# rest of the rows
def split(m):
    max_sum_column = m.max().max()
    #max_sum_column is the index of the splitting var
    # split function return F0, F1 or g0, g1
    f0=m[m[:,max_sum_column]==0]
    f1=m[m[:,max_sum_column]!=0]
    f1[:,max_sum_column]=0
    return f0,f1

def check_dim(L,R):
    return (len(L)>1 and len(R)>1)

dual_matrix=[]
def print_dual(L,R):
    if len(L)==1:
        dual_matrix=np.zeros((len(f.columns)-1,len(f.columns)-1))      
        for index, i in enumerate(f.iloc[0]):
            print(i)
            if i==1:
                dual_matrix[index][index]=1
        return dual_matrix
                
        
# e.g.: error_path = [path_0, path_1] = [['L', 'R'], ['L','L','L']]
#error_path=[]
def fk(L, R, path):
    if check_pre(L,R) == False:
        #error_path.append(path) 
        return
    if check_dim(L,R)==False:
        print(print_dual(L,R))
        
    else:
        f0,f1=split(L)
        g0,g1=split(R)
        L=np.concatenate((f0, f1),axis=0)
        R=g1
        print(f"path: {path}")
        path.append("L")
        fk(L, R, path)
        L=np.concatenate((g0,g1), axis=0)
        R=f1
        path = path[0:-2]
        path.append("R")
        fk(L, R, path)
#     return error_path
    print("it is not dual")